In [21]:
import openrouteservice
from openrouteservice import convert
import folium
import polyline
import geopy.distance

In [23]:
def get_route_between_locations(start_coords, end_coords, api_key):
    """
    Retrieve route between two locations with detailed debugging.
    
    Args:
    start_coords (tuple): (latitude, longitude) of start point
    end_coords (tuple): (latitude, longitude) of end point
    api_key (str): OpenRouteService API key
    
    Returns:
    dict: Route details or None if error
    """
    # Reorder coordinates to (lon, lat) for OpenRouteService
    start = [start_coords[1], start_coords[0]]
    end = [end_coords[1], end_coords[0]]
    
    # Calculate distance
    distance = geopy.distance.distance(start_coords, end_coords).kilometers
    print(f"Route distance: {distance:.2f} km")
    
    client = openrouteservice.Client(key=api_key)
    
    try:
        # Try different routing profiles
        profiles = ['driving-car', 'driving-hgv', 'cycling-regular']
        
        for profile in profiles:
            try:
                route = client.directions(
                    coordinates=[start, end],
                    profile=profile,
                    format='geojson'
                )
                
                route_info = {
                    'distance': route['features'][0]['properties']['summary']['distance'] / 1000,  # convert to km
                    'duration': route['features'][0]['properties']['summary']['duration'] / 60,  # convert to minutes
                    'profile': profile,
                    'geometry': route['features'][0]['geometry']
                }
                
                print(f"Successfully routed using {profile} profile")
                print(f"Route distance: {route_info['distance']:.2f} km")
                print(f"Route duration: {route_info['duration']:.2f} minutes")
                
                return route_info
            
            except openrouteservice.exceptions.ApiError as profile_error:
                print(f"Failed with {profile} profile: {profile_error}")
        
        return None
    
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [28]:
def visualize_route(route_details):
    """
    Visualize the route using Folium with GeoJSON coordinates.
    
    Args:
    route_details (dict): Route information from get_route_between_locations function
    
    Returns:
    folium.Map: Interactive map with route visualization
    """
    # Extract coordinates from GeoJSON
    if route_details['geometry']['type'] == 'LineString':
        decoded_coords = route_details['geometry']['coordinates']
        # Swap lon/lat to lat/lon for Folium
        decoded_coords = [[coord[1], coord[0]] for coord in decoded_coords]
    else:
        raise ValueError("Unsupported geometry type")
    
    # Create map centered on the route
    center_lat = sum(coord[0] for coord in decoded_coords) / len(decoded_coords)
    center_lon = sum(coord[1] for coord in decoded_coords) / len(decoded_coords)
    route_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)
    
    # Add route line
    folium.PolyLine(
        locations=decoded_coords, 
        color='blue', 
        weight=5, 
        opacity=0.8
    ).add_to(route_map)
    
    # Add start and end markers
    folium.Marker(
        location=decoded_coords[0], 
        popup='Start', 
        icon=folium.Icon(color='green', icon='play')
    ).add_to(route_map)
    
    folium.Marker(
        location=decoded_coords[-1], 
        popup='End', 
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(route_map)
    
    return route_map

In [24]:
start = (28.560212414670264, 77.1627974510193)
end = (28.56922, 77.30079)
api_key = "5b3ce3597851110001cf62482b5fcfdeefe14d60995ced782d556eca"

In [26]:
route = get_route_between_locations(start, end, api_key)

Route distance: 13.54 km
Failed with driving-car profile: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 77.3007900 28.5692200.'}, 'info': {'engine': {'build_date': '2024-12-02T11:09:21Z', 'graph_version': '1', 'version': '9.0.0'}, 'timestamp': 1737888909445}})
Failed with driving-hgv profile: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 77.3007900 28.5692200.'}, 'info': {'engine': {'build_date': '2024-12-02T11:09:21Z', 'graph_version': '1', 'version': '9.0.0'}, 'timestamp': 1737888909684}})
Successfully routed using cycling-regular profile
Route distance: 22.96 km
Route duration: 78.09 minutes


In [29]:
visualize_route(route)